In [1]:
state = 'WV' 
year = 2020
obj_type = 'average_Polsby_Popper'
starting_deviation = 0.01 

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of WV has 2 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from WV_plans_2020 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 160 plans from file.


In [6]:
from pareto import filter_and_sort_pareto
from metrics import scores

plans_scores = [scores(G, plan, G._ideal_population, obj_type) for plan in warm_starts]
_,_,nondominated_warm_starts_plans = filter_and_sort_pareto(plans=warm_starts, upper_bounds=plans_scores, obj_type=obj_type)
print(f"Selected {len(nondominated_warm_starts_plans)} nondominated warm start plans")

Selected 16 nondominated warm start plans


In [ ]:
from epsilon_constraint import epsilon_constraint_method

(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity = 'lcut',                                             # {'lcut', 'scf', 'shir'} 
            cutoff=None,       
            verbose= True,
            warm_start_mode = 'user',                                        # {'None', 'user', 'refinement'}
            warm_starts=warm_starts,                                         # if you have user define warm starts else it is None
            starting_deviation=starting_deviation, 
            time_limit=7200, 
            sizes=None,      
            max_B=True,                                                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True   
            symmetry_breaking='orbitope',                                    # {None, 'orbitope', 'rsum'} 
            state=state,
            year=year
        )

Initially, L = 887890 and U = 905826 and k = 2.

****************************************
Trying deviation = 8968.58
****************************************
Using user-provided warm starts.
Selected warm_start = [[0, 1, 2, 4, 6, 8, 13, 17, 18, 19, 20, 22, 23, 26, 27, 28, 29, 31, 33, 34, 36, 43, 45, 49, 50, 52, 54], [3, 5, 7, 9, 10, 11, 12, 14, 15, 16, 21, 24, 25, 30, 32, 35, 37, 38, 39, 40, 41, 42, 44, 46, 47, 48, 51, 53]]
Objective value: 0.23758746312472134
Deviation: 2230.0

****************************************
Running labeling model!
****************************************
L = 887890 and U = 905826
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Applying warm start!
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to val

* 1823    10              19       0.2387686    0.26983  13.0%  77.9    6s
* 1827    10              22       0.2389669    0.26983  12.9%  77.8    6s
* 1836    10              24       0.2389688    0.26983  12.9%  77.4    6s
* 1851    10              31       0.2392588    0.26983  12.8%  76.8    6s
* 1853    10              32       0.2392592    0.26983  12.8%  76.8    6s
H 1948     8                       0.2394748    0.23999  0.21%  73.8    6s

Cutting planes:
  Gomory: 1
  Cover: 10
  Implied bound: 3
  MIR: 42
  Flow cover: 17
  Inf proof: 25
  Zero half: 5
  RLT: 106

Explored 2033 nodes (146151 simplex iterations) in 6.84 seconds (2.20 work units)
Thread count was 20 (of 20 available processors)

Solution count 9: 0.239475 0.239259 0.239259 ... 0.18997
No other solutions better than 0.239475

Optimal solution found (tolerance 0.00e+00)
Best objective 2.394747734070e-01, best bound 2.394747734070e-01, gap 0.0000%

User-callback calls 4824, time in user-callback 0.12 sec

*********

  RHS range        [1e+00, 9e+05]

User MIP start produced solution with objective 0.189647 (0.03s)
User MIP start produced solution with objective 0.220897 (0.04s)
User MIP start produced solution with objective 0.221448 (0.05s)
Loaded user MIP start with objective 0.221448

Presolve removed 938 rows and 650 columns
Presolve time: 0.02s
Presolved: 813 rows, 736 columns, 3060 nonzeros
Presolved model has 2 quadratic constraint(s)
Variable types: 46 continuous, 690 integer (690 binary)

Root relaxation: objective 9.999990e-01, 264 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0   18    0.22145    1.00000   352%     -    0s
     0     0    0.97589    0   80    0.22145    0.97589   341%     -    0s
     0     0    0.97361    0   84    0.22145    0.97361   340%     -    0s
     0     0    0.94215    0   98    0.22145


Cutting planes:
  Lazy constraints: 5

Explored 47065 nodes (1361194 simplex iterations) in 59.71 seconds (9.19 work units)
Thread count was 20 (of 20 available processors)

Solution count 10: 0.215024 0.214965 0.214963 ... 0.21179
No other solutions better than 0.215024

Optimal solution found (tolerance 0.00e+00)
Best objective 2.150235176086e-01, best bound 2.150235176086e-01, gap 0.0000%

User-callback calls 96427, time in user-callback 0.76 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
plan = [[0, 1, 2, 4, 5, 6, 8, 9, 13, 17, 18, 19, 20, 22, 23, 26, 28, 29, 31, 33, 34, 36, 43, 45, 49, 50, 52, 54], [3, 7, 10, 11, 12, 14, 15, 16, 21, 24, 25, 27, 30, 32, 35, 37, 38, 39, 40, 41, 42, 44, 46, 47, 48, 51, 53]]

****************************************
Trying deviation = 1448.75
****************************************
Using user-provided warm starts.
Selected warm_start = [[0, 1, 2, 6, 9, 13, 17, 19, 20, 2


     0     0    1.00000    0   12    0.20025    1.00000   399%     -    0s
     0     0    0.97638    0   81    0.20025    0.97638   388%     -    0s
     0     0    0.97346    0   87    0.20025    0.97346   386%     -    0s
     0     0    0.97310    0   85    0.20025    0.97310   386%     -    0s
     0     0    0.91760    0  109    0.20025    0.91760   358%     -    0s
     0     0    0.90549    0   85    0.20025    0.90549   352%     -    0s
     0     0    0.88186    0  117    0.20025    0.88186   340%     -    0s
     0     0    0.88014    0  151    0.20025    0.88014   340%     -    0s
     0     0    0.87587    0  192    0.20025    0.87587   337%     -    1s
     0     0    0.87519    0  194    0.20025    0.87519   337%     -    1s
     0     0    0.87078    0  209    0.20025    0.87078   335%     -    1s
     0     0    0.86942    0  217    0.20025    0.86942   334%     -    1s
     0     0    0.86218    0  212    0.20025    0.86218   331%     -    1s
     0     0    0.86212 

...

 137348 28322 infeasible   64         0.17845    0.21621  21.2%  23.8  145s
 143776 29576    0.21521   46   69    0.17845    0.21592  21.0%  23.9  151s
 149880 30579 infeasible   39         0.17845    0.21540  20.7%  23.9  156s
 156671 31049 infeasible   50         0.17845    0.21484  20.4%  24.0  160s
 164785 31321    0.20274   59   22    0.17845    0.21419  20.0%  24.1  166s
 168277 31584    0.20205   53   25    0.17845    0.21353  19.7%  24.2  170s
 172936 31996    0.19714   58  126    0.17845    0.21156  18.6%  24.2  175s
 182192 32334    0.19531   48   17    0.17845    0.21094  18.2%  24.2  180s
 188255 32324 infeasible   56         0.17845    0.21049  18.0%  24.2  185s
 195906 30709 infeasible   67         0.17845    0.20918  17.2%  24.2  190s
 206815 29321 infeasible   65         0.17845    0.20501  14.9%  24.2  195s
 212192 30400    0.20306   48   65    0.17845    0.20312  13.8%  24.3  201s
 218064 31633    0.20294   56   26    0.17845    0.20312  13.8%  24.4  207s
 221899 3226

User MIP start produced solution with objective 0.139421 (0.04s)
User MIP start produced solution with objective 0.148211 (0.05s)
Loaded user MIP start with objective 0.148211

Presolve removed 939 rows and 650 columns
Presolve time: 0.03s
Presolved: 812 rows, 736 columns, 3013 nonzeros
Presolved model has 2 quadratic constraint(s)
Variable types: 46 continuous, 690 integer (690 binary)

Root relaxation: objective 9.999990e-01, 258 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0   18    0.14821    1.00000   575%     -    0s
     0     0    0.97587    0   76    0.14821    0.97587   558%     -    0s
     0     0    0.97317    0   82    0.14821    0.97317   557%     -    0s
     0     0    0.97313    0   82    0.14821    0.97313   557%     -    0s
     0     0    0.94662    0  126    0.14821    0.94662   539%     - 

 266817 76391    0.19139   75   96    0.14821    0.19901  34.3%  34.7  370s
 271565 77045 infeasible   77         0.14821    0.19882  34.1%  34.6  375s
 277868 78228    0.15735   80   62    0.14821    0.19852  33.9%  34.5  382s
 281529 78866    0.17032   72   23    0.14821    0.19836  33.8%  34.5  386s
 285895 79654    0.16274   80   28    0.14821    0.19813  33.7%  34.4  390s
 289370 80112    0.17699   83   49    0.14821    0.19793  33.5%  34.4  395s
 293148 80811 infeasible   83         0.14821    0.19775  33.4%  34.4  403s
 295890 80896    0.17187   74   48    0.14821    0.19757  33.3%  34.4  405s
 300120 81620    0.16888   79   15    0.14821    0.19735  33.2%  34.3  411s
 303733 81810    0.19228   80   45    0.14821    0.19717  33.0%  34.3  416s
 308013 82490     cutoff   77         0.14821    0.19698  32.9%  34.2  422s
 310168 82502 infeasible   69         0.14821    0.19684  32.8%  34.2  425s
 314189 83310 infeasible   84         0.14821    0.19663  32.7%  34.2  430s
 317932 8363

 673498 154299    0.18209   91   36    0.14821    0.18671  26.0%  34.2  911s
 678342 155953    0.16761   74   89    0.14821    0.18669  26.0%  34.2  916s
 683603 157625    0.17187   77   23    0.14821    0.18667  25.9%  34.1  922s
 688596 159367    0.15965   86   39    0.14821    0.18664  25.9%  34.1  926s
 693582 161191    0.18524   83   30    0.14821    0.18663  25.9%  34.0  931s
 698522 162921    0.18184   66   47    0.14821    0.18660  25.9%  34.0  938s
 701701 163551 infeasible   75         0.14821    0.18660  25.9%  34.0  941s
 706546 165312    0.18452   80   49    0.14821    0.18658  25.9%  34.0  946s
 711337 166989    0.18180   78   36    0.14821    0.18656  25.9%  34.0  952s
 713319 168180    0.18550   62   44    0.14821    0.18655  25.9%  34.0  955s
 718499 170025    0.15358   77   31    0.14821    0.18653  25.9%  33.9  962s
 721852 170585     cutoff   76         0.14821    0.18652  25.8%  33.9  965s
 726785 172289    0.15375   81   51    0.14821    0.18650  25.8%  33.9  971s

 1088255 288749     cutoff   81         0.14821    0.18554  25.2%  32.4 1445s
 1092943 290231 infeasible   85         0.14821    0.18554  25.2%  32.4 1452s
 1094838 291291    0.17503   82   37    0.14821    0.18553  25.2%  32.4 1456s
 1100221 292811    0.18309   79   69    0.14821    0.18552  25.2%  32.4 1463s
 1103259 293228    0.15167   82   34    0.14821    0.18552  25.2%  32.4 1465s
 1107983 294694    0.18012   69   47    0.14821    0.18551  25.2%  32.4 1470s
 1111843 296333    0.17083   80   69    0.14821    0.18550  25.2%  32.4 1477s
 1114866 296817    0.15498   89   22    0.14821    0.18549  25.2%  32.4 1481s
 1116577 297806    0.18133   86   46    0.14821    0.18549  25.2%  32.4 1485s
 1121714 298781 infeasible   76         0.14821    0.18548  25.1%  32.4 1490s
 1126565 300188    0.18484   83   68    0.14821    0.18547  25.1%  32.3 1497s
 1131685 301762    0.18127   84   93    0.14821    0.18546  25.1%  32.3 1503s
 1133465 302324    0.18537   76   66    0.14821    0.18546  25.1

In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
epsilon = 1 / (2 * G._k)
min_deviation = min(round(r[2], 1) for r in result)

if min_deviation < epsilon:
    no_solution_region = None
else:
    no_solution_region = [0, min_deviation]

if no_solution_region is not None:
    print(f"No feasible solution was found within the region: {no_solution_region}")

In [ ]:
from pareto import plot_pareto_frontiers

plot_pareto_frontiers(
                G,
                method='epsilon_constraint_method',
                plans=None,                                   #if method ='epsilon_constraint_method' is None 
                obj_types=obj_type,                               
                ideal_population=G._ideal_population,
                state=state,
                filepath=filepath,
                filename2=filename2,
                no_solution_region=no_solution_region,
                year=year,
                result=result                               #if method ='heuristic' is None 
             )

In [ ]:
from draw import draw_plan
from metrics import observed_deviation_persons, compute_obj

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: int(x)}
G._L = 0
G._U = G._k * G._ideal_population

for plan, obj_bound, dev in result:
    obs_dev = observed_deviation_persons(G, plan, G._ideal_population)
    obj = compute_obj(G, plan, obj_type)
    obj_val = format_obj.get(obj_type, lambda x: round(x, 4))(obj)
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    title = f"{round(obs_dev, 2)}-person deviation ({deviation_percentage}%), with {obj_val} {obj_type}"
    draw_plan(filepath, filename2, G, plan, title=title, year=year)